In [1]:
import cv2
import math
import torch
import pandas as pd
#from collections import defaultdict
from ultralytics import YOLO

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Lenovo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-27 Python-3.9.13 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
model.classes=[2]

In [4]:
video_path = 'cars.mp4'

trackers = []
car_centers = {}
next_car_id = 0 

cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    current_car_centers = []
    
    for result in results.xyxy[0]: 
        x1, y1, x2, y2, conf, cls = result

        if int(cls) == 2 and conf > 0.5: 
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            center = int((x1 + x2) / 2), int((y1 + y2) / 2)
            current_car_centers.append(center)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'Car', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    new_car_centers = []
    for center in current_car_centers:
        matched = False
        for car_id, old_center in car_centers.items():
            if math.sqrt((center[0] - old_center[0]) ** 2 + (center[1] - old_center[1]) ** 2) < 50:
                car_centers[car_id] = center
                new_car_centers.append((car_id, center))
                matched = True
                break
        if not matched:
            car_centers[next_car_id] = center
            new_car_centers.append((next_car_id, center))
            next_car_id += 1

    car_centers = {car_id: center for car_id, center in new_car_centers}

    for car_id, center in car_centers.items():
        cv2.putText(frame, f'ID: {car_id}', (center[0] - 10, center[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    out.write(frame)

    cv2.imshow("Car Detection and Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()